In [1]:
library(readxl)
library(here)
library(dplyr)
library(data.table)
library(xtable)
library(vars)
library(forecast)

Warning message:
"package 'here' was built under R version 3.6.3"here() starts at C:/Users/Daniel/Desktop/Daniel/codes/python_R/FGV_Financial_Econometrics

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

Warning message:
"package 'vars' was built under R version 3.6.3"Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select

Loading required package: strucchange
Warning message:
"package 'strucchange' was built under R version 3.6.3"Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich
Warning message:
"package 'sandwich' was built under R

### Base de dados

Um dos indicadores mais importantes dos estados unidos é o "Institute of Supply Management (ISM) Manufacturing Purchasing Managers Index (PMI)", mais conhecido no mercado como ISM. Este relatorio é baseado em dados copilados de respotas mensais a executivos do lado da demanda e da oferta em mais de 400 companias.

Um fato importante para os agentes que tentam antecipar o ISM é que muitos estados reportam antecipadamente seus ISM. Portanto, é comum utilizar os ISMs estaduais como variáveis independentes no modelo.

### Objetivo

O objetivo principal deste projeto será implementar a metodologia de combinação de previsão proposta por Bates e Granger (1989) utilizando uma série de metodos de previsão no contexto de séries temporais. Mais especificamente, testarermos os seguintes modelos:

> **(1)** ARIMA

> **(2)** ARIMAX

> **(3)** Regressão linear com erros gaussianos

> **(4)** VAR

Uma vez em posse das estimações dentro da amostra destes modelos, utilizaremos o procedimento de previsão recursiva para avaliar, utiliando a metrica "mean squared erro (MSE)", os erros de previsão de cada um dos modelos individualmente, de uma combinação por média, e da combinação proposta por Bates e Granger.

In [2]:
df = readxl::read_excel(here('src', 'data', 'ISM.xlsx'))
df = df[, c('date', 'ISM_Head', 'TX_ISM', 'KAN_ISM', 'EMP_ISM', 'PHI_ISM', 'RICH_ISM')]
df = df[complete.cases(df), ]

head(df)

date,ISM_Head,TX_ISM,KAN_ISM,EMP_ISM,PHI_ISM,RICH_ISM
2006-01-31,55.0,30.9,16,17.5,5.4,6
2006-02-28,55.8,29.0,12,20.9,15.3,2
2006-03-31,54.3,32.9,17,33.2,14.7,14
2006-04-30,55.2,25.8,15,18.5,14.2,7
2006-05-31,53.7,18.5,18,18.6,13.7,-4
2006-06-30,52.0,15.4,18,26.5,11.7,-3


In [3]:
tail(df)

date,ISM_Head,TX_ISM,KAN_ISM,EMP_ISM,PHI_ISM,RICH_ISM
2020-05-31,43.1,-49.2,-19,-48.5,-43.1,-28
2020-06-30,52.6,-6.1,1,-0.2,27.5,0
2020-07-31,54.2,-3.0,3,17.2,24.1,10
2020-08-31,56.0,8.0,14,3.7,17.2,18
2020-09-30,55.4,13.6,11,17.0,15.0,21
2020-10-31,59.3,19.8,13,10.5,32.3,29


In [4]:
df_est = df[1:100,] %>% dplyr::select(-date)
df_fore = df[101:dim(df)[1],] %>% dplyr::select(-date)

### ARIMA(p, j, q)

Como visto em aula, utilizaremos os criterios de informacao para determinar os termos $p$ e $q$.

In [5]:
ism_arma = list('AIC' = data.table(), 'BIC' = data.table())
for (ar.lag in 0:10) {
  arma.stat = rep(0, 6)
  for (ma.lag in 0:2) {
    arma.fit = arima(df_est[,c('ISM_Head')], order = c(ar.lag, 0, ma.lag))
    # arma.fit
    # AIC
    arma.stat[ma.lag + 1] <- arma.fit$aic
    # BIC
    arma.stat[ma.lag + 4] <- -2 * arma.fit$loglik + (ar.lag + ma.lag) * log(length(df_est[,c('ISM_Head')]))
  }
  ism_arma$AIC = rbindlist(list(ism_arma$AIC, data.table(t(arma.stat[1:3]))))
  ism_arma$BIC = rbindlist(list(ism_arma$BIC, data.table(t(arma.stat[4:6]))))
}
setnames(ism_arma$AIC, c('MA0', 'MA1', 'MA2'))
ism_arma$AIC[, AR := 0:10]
setnames(ism_arma$BIC, c('MA0', 'MA1', 'MA2'))
ism_arma$BIC[, AR := (0:10)]


BIC_selec.mat = rbind(ism_arma$BIC[, AR := (0:10)])
print(xtable(BIC_selec.mat))

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% Sun Dec 06 16:13:56 2020
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & MA0 & MA1 & MA2 & AR \\ 
  \hline
1 & 604.33 & 518.25 & 467.59 &   0 \\ 
  2 & 414.96 & 414.54 & 409.52 &   1 \\ 
  3 & 414.33 & 414.91 & 407.38 &   2 \\ 
  4 & 408.48 & 407.21 & 408.55 &   3 \\ 
  5 & 407.30 & 407.29 & 407.09 &   4 \\ 
  6 & 407.30 & 404.85 & 403.99 &   5 \\ 
  7 & 406.62 & 406.57 & 401.51 &   6 \\ 
  8 & 406.53 & 406.49 & 401.82 &   7 \\ 
  9 & 406.43 & 405.99 & 398.26 &   8 \\ 
  10 & 406.38 & 405.84 & 400.73 &   9 \\ 
  11 & 406.07 & 405.18 & 395.81 &  10 \\ 
   \hline
\end{tabular}
\end{table}


A combinação que minimiza o criterio BIC é ar(10) e ma(2), portanto este será nossa especificação do modelo ARMA

### ARIMAX(p, j, q)

O modelo arimax é simplesmente o modelo modelo arima com variáveis exogenas. Utilizaremos uma especificação mais simples do que a encontrada anteriormente mas colocaremos todas os outros ISM como variaveis exógenas.

### O modelo de regressão linear com erros gaussianos

Mais simples de todos. Utilizaremos o modelos de regressão tradicional com erros gaussianos e constante, além de usar todos os outros ISM como variáveis exógenas.

### VAR

Utilizaremos o mesmo procedimento de especificação da ordem $p$ do VAR(p) utilizado no modelo ARMA para determinar quantos termos autoregressivos utilizaremos na regressão.

In [6]:
VARselect(df_est, lag.max = 5)

,1,2,3,4,5
AIC(n),1.990636e+01,2.001854e+01,2.001364e+01,2.028666e+01,2.030054e+01
HQ(n),2.036259e+01,2.086583e+01,2.125199e+01,2.191607e+01,2.232101e+01
SC(n),2.103544e+01,2.211540e+01,2.307829e+01,2.431910e+01,2.530077e+01
FPE(n),4.425056e+08,4.993920e+08,5.082238e+08,6.973111e+08,7.602428e+08


De acordo com todos os critérios, inclusive o AIC, o número de termos autoregressivos $p$ que os minimiza é 1. 

### Função de previsão fora da amostra

In [8]:
pred_list = list()
df_oos = df %>% dplyr::select(-date)
j = 1
for (i in 100:dim(df_oos)[1]){
    comb = list()
    modd_arima = Arima(y = df_oos[1:i, c('ISM_Head')], order = c(10, 0, 2))
    pred_arima = predict(modd_arima, n.ahead = 1)
    pred_list[['arima']][j] = pred_arima$pred[1]
    comb[1] = pred_arima$pred[1]
    
    
    modd_arimax = Arima(y = df_oos[1:i, c('ISM_Head')], xreg = as.matrix(df_oos[1:i, c('TX_ISM', 'KAN_ISM', 'EMP_ISM', 'PHI_ISM', 'RICH_ISM')]),
                       order = c(1, 0, 1))
    pred_arimax = predict(modd_arimax, newxreg = as.matrix(df_oos[i+1, c('TX_ISM', 'KAN_ISM', 'EMP_ISM', 'PHI_ISM', 'RICH_ISM')]),
                         n.ahead = 1)
    pred_list[['arimax']][j] = pred_arimax$pred[1]
    comb[2] = pred_arimax$pred[1]

    
    modd_lm = lm(ISM_Head ~TX_ISM + KAN_ISM + EMP_ISM + PHI_ISM + RICH_ISM,  df_oos[1:i, ])
    pred_lm = predict(modd_lm, newxreg = df_oos[i+1, c('TX_ISM', 'KAN_ISM', 'EMP_ISM', 'PHI_ISM', 'RICH_ISM')],
                     n.ahead = 1)
    pred_list[['lm']][j] = pred_lm[[1]]
    comb[3] = pred_lm[[1]]

    
    modd_var = VAR(df_oos[1:i, c('ISM_Head', 'TX_ISM', 'KAN_ISM', 'EMP_ISM', 'PHI_ISM', 'RICH_ISM')], p=1)
    pred_var = predict(modd_var, n.ahead = 1)
    pred_list[['var']][j] = pred_var$fcst$ISM_Head[1]
    comb[4] = pred_var$fcst$ISM_Head[1]

    pred_mean = mean(do.call('rbind', comb), na.rm=TRUE)
    pred_list[['mean']][j] = pred_mean
    #pred_bates_granger
    j = j+1
}

In [12]:
df_final = do.call('cbind', pred_list)

In [27]:
install.packages("C:/Users/Daniel/Downloads/Rcpp_1.0.5.zip", repos = NULL)

package 'Rcpp' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'Rcpp'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Daniel\Anaconda3\envs\Renv\Lib\R\library\00LOCK\Rcpp\libs\x64\Rcpp.dll to C:\Users\Daniel\Anaconda3\envs\Renv\Lib\R\library\Rcpp\libs\x64\Rcpp.dll: Permission denied"Warning message:
"restored 'Rcpp'"

In [32]:
library("pacman")

Warning message:
"package 'pacman' was built under R version 3.6.3"

In [33]:
p_unlock()


No 00LOCK detected in:
 C:/Users/Daniel/Anaconda3/envs/Renv/Lib/R/library


In [30]:
Sys.getenv("R_LIBS_USER")

[1] "C:\\Users\\Daniel\\Documents/R/win-library/3.6"

In [29]:
install.packages("Rcpp")

package 'Rcpp' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'Rcpp'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Daniel\Anaconda3\envs\Renv\Lib\R\library\00LOCK\Rcpp\libs\x64\Rcpp.dll to C:\Users\Daniel\Anaconda3\envs\Renv\Lib\R\library\Rcpp\libs\x64\Rcpp.dll: Permission denied"Warning message:
"restored 'Rcpp'"


The downloaded binary packages are in
	C:\Users\Daniel\AppData\Local\Temp\RtmpYlZ3FI\downloaded_packages


In [ ]:
library('GeomComb')